# Imports

In [1]:
import pandas as pd
import os
from nltk.corpus import stopwords
import nltk
import numpy as np

In [2]:
def saveCSV(dataframe, filename):
    dataframe.to_csv("C:/Users/Jan/Documents/Python_Projects/Bachelorthesis/Bachelorthesis/Analysis/DataFrames/"+ filename +".csv",index=False)

# Load data

In [3]:
# Folderpath to CSV's
CSV_folderPath="C:/Users/Jan/Documents/Python_Projects/Bachelorthesis/Bachelorthesis/CSV_Backlog"

# Initialize list<csv>
Combi_csvs = []

# Change directory to CSV_folderPath
os.chdir(CSV_folderPath)

# iterate over all files
for file in os.listdir():
    # that are csvs...
    if file.endswith(".csv"):
        print(file)
        # read csv to dataframe
        df =pd.read_csv(file, index_col=None, header=0,encoding = "utf-8")
        # all df append to combi...
        Combi_csvs.append(df)
All_articles = pd.concat(Combi_csvs, axis=0, ignore_index=True)

Bild_01012022_13_20_40.csv
Bild_01022022_13_41_10.csv
Bild_01032022_08_07_43.csv
Bild_01042022_08_50_35.csv
Bild_01052022_12_11_05.csv
Bild_01062022_12_11_10.csv
Bild_01072022_12_18_51.csv
Bild_01122021_11_38_36.csv
Bild_02012022_13_20_07.csv
Bild_02022022_12_04_19.csv
Bild_02032022_10_06_55.csv
Bild_02042022_12_44_15.csv
Bild_02052022_12_10_12.csv
Bild_02062022_12_10_22.csv
Bild_02072022_12_18_20.csv
Bild_02122021_12_17_28.csv
Bild_03022022_13_09_02.csv
Bild_03032022_21_37_11.csv
Bild_03042022_10_59_34.csv
Bild_03052022_12_10_11.csv
Bild_03062022_12_10_33.csv
Bild_03072022_12_18_47.csv
Bild_03122021_12_02_05.csv
Bild_03122021_12_05_39.csv
Bild_04022022_12_57_13.csv
Bild_04032022_08_45_29.csv
Bild_04042022_09_48_01.csv
Bild_04052022_12_10_33.csv
Bild_04062022_12_10_39.csv
Bild_04072022_12_18_25.csv
Bild_04122021_12_44_20.csv
Bild_05022022_15_18_23.csv
Bild_05032022_09_13_22.csv
Bild_05042022_08_24_11.csv
Bild_05052022_12_10_33.csv
Bild_05062022_12_10_50.csv
Bild_05072022_12_19_09.csv
B

## Data Cleaning

In [4]:
# lower categories
All_articles["Kategorie"] = All_articles["Kategorie"].str.lower()

# cast date info
All_articles["Date_Info"] = All_articles["Date_Info"].str.strip()

All_articles["Date_Info"] = pd.to_datetime(All_articles["Date_Info"], format='%d%m%Y_%H:%M', errors="coerce").fillna(pd.to_datetime(All_articles["Date_Info"], format="%Y-%m-%d %H:%M:%S", errors="coerce").fillna(pd.to_datetime(All_articles["Date_Info"], format="%d%m%Y_%H:%M", errors="coerce")))

# Drop duplicates
All_articles= All_articles.drop_duplicates(subset="Titel")
All_articles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 206363 entries, 0 to 446606
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Titel                  206362 non-null  object        
 1   Overline               175352 non-null  object        
 2   Date_Info              206361 non-null  datetime64[ns]
 3   Kategorie              203490 non-null  object        
 4   URL                    206363 non-null  object        
 5   detailed_informations  84488 non-null   object        
 6   bild_plus              48632 non-null   object        
 7   Zugriff_Datum          179168 non-null  object        
 8   News_page              206363 non-null  object        
 9   author                 32981 non-null   object        
 10  Breadcrumb             88523 non-null   object        
dtypes: datetime64[ns](1), object(10)
memory usage: 18.9+ MB


## Only use data after 10.01.2022 because there was a lag of data before


In [109]:
#greater than the start date and smaller than the end date
start_date = pd.to_datetime("2022 1 10")
end_date = pd.to_datetime('today')

All_articles = All_articles.loc[All_articles["Date_Info"].between(start_date, end_date)]

## Ammount of articles


In [103]:
# Spiegel articles
ammount_of_Spiegel_articles = len(All_articles.loc[All_articles['News_page'] == "Spiegel"])
print(f"Spiegel articles: {ammount_of_Spiegel_articles}")

# Sueddeutsche articles
ammount_of_Sueddeutsche_articles = len(All_articles.loc[All_articles['News_page'] == "Sueddeutsche"])
print(f"Sueddeutsche articles: {ammount_of_Sueddeutsche_articles}")

# Bild articles
ammount_of_Bild_articles = len(All_articles.loc[All_articles['News_page'] == "Bild"])
print(f"Bild articles: {ammount_of_Bild_articles}")

# All articles
ammount_of_Combined_articles = len(All_articles)
print(f"Overall: {ammount_of_Combined_articles}")

Spiegel articles: 20314
Sueddeutsche articles: 120192
Bild articles: 39821
Overall: 180327


## Merge Titel and Overline
Titel und Overline werden nun zusammengefügt um die Thematik des Artikels besser herauszuarbeiten

In [104]:
All_articles["Titel"] = np.where(All_articles["News_page"] == "Bild", All_articles["Titel"] + " " + All_articles["Overline"],All_articles["Titel"])
All_articles["Titel"] = All_articles["Titel"].astype(str)
saveCSV(All_articles,"All_news_articles")

## Segmentation
Segmentierung von Titel_and_Overline

1. Step
Create new column with Titel_and_Overline as list

In [79]:
All_articles["Tokens"] = All_articles.apply(lambda row: nltk.word_tokenize(row['Titel']), axis=1)

2. Step
Explode dataframe on Title_and_Overline_list

In [80]:
All_articles_workingCopy = All_articles.copy()
All_articles = All_articles_workingCopy.explode("Tokens")

## Remove stopwords

In [81]:
All_articles = All_articles[~All_articles['Tokens'].isin(stopwords.words("german"))]
# more stopwords
# continues extension
uninteresting_titles= ["Der", "Die", "Das","Was","Warum","Er","Sie", "Es", "Ich", "Du", "Mit","Wie","Ein","So","Wir","Nach","Am","An","B","Bei","Beim","Bin","Bis","Co","Da","Hier","Diese","Darum","Mehr","Jetzt","Wer","A","C",":","»","«", "," ,"\'\'","``", "_","-" ".","?","–", "-",".","!","(",")"]
All_articles=All_articles[~All_articles.Tokens.isin(uninteresting_titles)]

## Remove words that start with lower

In [82]:
All_articles = All_articles[All_articles["Tokens"].str.istitle()]

## Remove char "-" before and after keyword

In [83]:
All_articles["Tokens"] = All_articles["Tokens"].apply(lambda x : x[1:] if x.startswith("-") else x)
All_articles["Tokens"] = All_articles["Tokens"].apply(lambda x : x[1:] if x.endswith("-") else x)

In [84]:
# ammount of words
len(All_articles["Tokens"])

693153

## Stemming
Wird nicht verwendet da es keine guten Ergebnisse erzielt

In [85]:
#stemmer = nltk.SnowballStemmer('german')
#All_articles["Tokens_stemmed"] = All_articles["Tokens"].apply(stemmer.stem)

## Create KeyWord list

In [87]:
keyWords=All_articles.groupby("Tokens").size()
keyWords = pd.DataFrame({"KeyWord":keyWords.index,"Occurence":keyWords.values})

,KeyWord,Occurence
0,'Besserkatholiken,1
1,'Die,1
2,'Du,1
3,'Hitler,1
4,'Ich,1
...,...,...
108979,›The,1
108980,›Titanic‹,1
108981,›Trash‹,1
108982,›Z‹,1


## Articles that occure more than n times

In [88]:
occurence = [0,10,25,50,75,100,125,150,200,300,350,400,450,500]
for n in occurence:
    keyWords_by_N = keyWords.loc[keyWords["Occurence"] >= n]
    print("n:"+str(n) +" = "+str(len(keyWords_by_N)))

n:0 = 108984
n:10 = 9729
n:25 = 4177
n:50 = 2063
n:75 = 1343
n:100 = 1002
n:125 = 769
n:150 = 616
n:200 = 423
n:300 = 220
n:350 = 175
n:400 = 149
n:450 = 115
n:500 = 92


In [89]:
saveCSV(All_articles,"All_news_articles_exploded")
saveCSV(keyWords,"keyWords")